In [1]:
import pandas as pd
import os
import glob
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import time
%pylab inline

//anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


Populating the interactive namespace from numpy and matplotlib


In [2]:
# This cell assumes you hav the five_min_frame.hdf which is a hdf output of a pandas dataframe
# of all 2015 data without the lane specific columns
# start_time = time.time()
# df = pd.read_hdf('five_min_frame.hdf','five_min_frame')
# print "total time: %s" % (time.time() - start_time)

total time: 108.548623085


In [3]:
# filter to main line stations only
# df_ml = df[df['Lane Type'] == 'ML']

In [4]:
# df_core = df_ml[['Timestamp', 'Station', 'Freeway #', 'Direction', '% Observed', 'Total Flow', 'Avg Occupancy',
#                  'Avg Speed']]
# df_core.to_hdf('2015_ml_core.hdf', 'ml_core', mode='w')

In [2]:
# If you have the 2015_ml_core.hdf file then start here
df_core = pd.read_hdf('2015_ml_core.hdf', 'ml_core')

In [3]:
no_dup_keep_last = pd.read_hdf('meta_2015.hdf', 'meta_2015')

In [4]:
to_loop = no_dup_keep_last.groupby(['Fwy', 'Dir'])['ID'].count().reset_index()[['Fwy', 'Dir']].values

In [5]:
daymap = {0:'Monday', 1:'Tuesday', 2:'Wednesday', 3:'Thursday', 4:'Friday', 5:'Saturday', 6:'Sunday'}

def add_date_time_weekday( df ):
    """
    Josh's add date_time function
    """
    df.loc[:,'Timestamp2'] = pd.to_datetime(df['Timestamp'], format='%m/%d/%Y %H:%M:%S')
    df.loc[:,'Time'] = df['Timestamp2'].dt.time
    df.loc[:,'Date'] = df['Timestamp2'].dt.date
    df.loc[:,'Weekday'] = df['Date'].apply( lambda x: daymap[x.weekday()] )
    return df

In [11]:
def create_freeway_day_pivot(Fwy, Dir):
    """
    This function is designed to pivot a particular freeway / direction
    """
    print "pivoting Fwy: %s Dir: %s" % (Fwy, Dir)
    weekdays = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']
    weekends = ['Saturday', 'Sunday']
    
    if Dir == "N":
        sort_order = ('Abs_PM', True)        
    elif Dir == "S":
        sort_order = ('Abs_PM', False)        
    elif Dir == "E":
        sort_order = ('Abs_PM', True)        
    elif Dir == "W":
        sort_order = ('Abs_PM', False) 

    ret = no_dup_keep_last[(no_dup_keep_last.Fwy == Fwy) & (no_dup_keep_last.Dir == Dir)]\
        .sort_values(by=sort_order[0], ascending=sort_order[1])
    ret.index = np.arange(0, ret.shape[0])
    station_list = list(ret.ID.values)
    stations = df_core.Station.isin(station_list)
    df_stations = df_core[stations]

    if df_stations.Timestamp.count() > 0:
        for col, f_name in [('Total Flow', 'Flow'), ('Avg Occupancy', 'Occu'), ('Avg Speed', 'Speed')]:
    #         for day in daymap.values():
    #             # print "day: %s" % day
    #             df = stations_with_date[stations_with_date.Weekday == day]

    #             # note: only creating flow at this point...
    #             pivoted = df.pivot_table(index='Station', columns='Time', values=col, aggfunc=np.mean)
    #             name = '%s_2015_%s_%s_%s' % (f_name, Fwy, Dir, day)
    #             pivoted.to_hdf('%s.hdf' % (name), name, mode='w')

            for partition, part_name in [(weekdays, 'Weekdays'), (weekends, 'Weekends')]:
                # print "part_name: %s" % part_name
                partition_df = stations_with_date[ stations_with_date.Weekday.isin(partition) ]
                pivoted = partition_df.pivot_table(index='Station', columns='Time', values=col, aggfunc=np.mean)
                name = '%s_2015_%s_%s_%s' % (f_name, Fwy, Dir, part_name)
                pivoted.to_hdf('%s.hdf' % (name), name, mode='w')
    else:
        print "Fwy: %s Dir: %s has no data" % (Fwy, Dir)

In [7]:
for Fwy, Dir in to_loop:
    start_time = time.time()
    stations_with_date = create_freeway_day_pivot(Fwy, Dir)
    print "time: %s" % (time.time() - start_time)

pivoting Fwy: 5 Dir: N


//anaconda/lib/python2.7/site-packages/pandas/core/indexing.py:288: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
//anaconda/lib/python2.7/site-packages/pandas/core/indexing.py:465: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
//anaconda/lib/python2.7/site-packages/pandas/io/pytables.py:260: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->time,key->axis0] [items->None]

  f(store)
//anacon

time: 293.240375996
pivoting Fwy: 5 Dir: S
time: 47.0048408508
pivoting Fwy: 8 Dir: E
time: 44.1970670223
pivoting Fwy: 8 Dir: W
time: 42.310585022
pivoting Fwy: 15 Dir: N
time: 43.1920349598
pivoting Fwy: 15 Dir: S
time: 43.387130022
pivoting Fwy: 52 Dir: E
time: 45.5881698132
pivoting Fwy: 52 Dir: W
time: 42.3144989014
pivoting Fwy: 54 Dir: E
time: 44.0053391457
pivoting Fwy: 54 Dir: W
time: 47.2775499821
pivoting Fwy: 56 Dir: E
time: 41.1542019844
pivoting Fwy: 56 Dir: W
time: 42.6294419765
pivoting Fwy: 67 Dir: N
time: 45.3795349598
pivoting Fwy: 67 Dir: S
time: 44.1995658875
pivoting Fwy: 78 Dir: E
time: 39.8686261177
pivoting Fwy: 78 Dir: W
time: 42.1908481121
pivoting Fwy: 94 Dir: E
time: 43.8916831017
pivoting Fwy: 94 Dir: W
time: 43.0892989635
pivoting Fwy: 125 Dir: N
time: 40.7641320229
pivoting Fwy: 125 Dir: S
time: 43.0152680874
pivoting Fwy: 163 Dir: N
time: 49.2900438309
pivoting Fwy: 163 Dir: S
time: 39.2246778011
pivoting Fwy: 805 Dir: N
time: 41.6067929268
pivoting Fwy

In [10]:
# List freeway weekday / weekends
!ls Flow_2015*Week*.hdf

Flow_2015_125_N_Weekdays.hdf Flow_2015_5_S_Weekdays.hdf
Flow_2015_125_N_Weekends.hdf Flow_2015_5_S_Weekends.hdf
Flow_2015_125_S_Weekdays.hdf Flow_2015_67_N_Weekdays.hdf
Flow_2015_125_S_Weekends.hdf Flow_2015_67_N_Weekends.hdf
Flow_2015_15_N_Weekdays.hdf  Flow_2015_67_S_Weekdays.hdf
Flow_2015_15_N_Weekends.hdf  Flow_2015_67_S_Weekends.hdf
Flow_2015_15_S_Weekdays.hdf  Flow_2015_78_E_Weekdays.hdf
Flow_2015_15_S_Weekends.hdf  Flow_2015_78_E_Weekends.hdf
Flow_2015_163_N_Weekdays.hdf Flow_2015_78_W_Weekdays.hdf
Flow_2015_163_N_Weekends.hdf Flow_2015_78_W_Weekends.hdf
Flow_2015_163_S_Weekdays.hdf Flow_2015_805_N_Weekdays.hdf
Flow_2015_163_S_Weekends.hdf Flow_2015_805_N_Weekends.hdf
Flow_2015_52_E_Weekdays.hdf  Flow_2015_805_S_Weekdays.hdf
Flow_2015_52_E_Weekends.hdf  Flow_2015_805_S_Weekends.hdf
Flow_2015_52_W_Weekdays.hdf  Flow_2015_8_E_Weekdays.hdf
Flow_2015_52_W_Weekends.hdf  Flow_2015_8_E_Weekends.hdf
Flow_2015_54_E_Weekdays.hdf  Flow_2015_8_W_Weekdays.hdf
Flow_2015_54_E_Weekends.hdf  Flo

In [12]:
!ls Occ*

Occu_2015_125_N_Weekdays.hdf Occu_2015_5_S_Weekdays.hdf
Occu_2015_125_N_Weekends.hdf Occu_2015_5_S_Weekends.hdf
Occu_2015_125_S_Weekdays.hdf Occu_2015_78_E_Weekdays.hdf
Occu_2015_125_S_Weekends.hdf Occu_2015_78_E_Weekends.hdf
Occu_2015_15_N_Weekdays.hdf  Occu_2015_78_W_Weekdays.hdf
Occu_2015_15_N_Weekends.hdf  Occu_2015_78_W_Weekends.hdf
Occu_2015_15_S_Weekdays.hdf  Occu_2015_805_N_Weekdays.hdf
Occu_2015_15_S_Weekends.hdf  Occu_2015_805_N_Weekends.hdf
Occu_2015_163_N_Weekdays.hdf Occu_2015_805_S_Weekdays.hdf
Occu_2015_163_N_Weekends.hdf Occu_2015_805_S_Weekends.hdf
Occu_2015_163_S_Weekdays.hdf Occu_2015_8_E_Weekdays.hdf
Occu_2015_163_S_Weekends.hdf Occu_2015_8_E_Weekends.hdf
Occu_2015_52_E_Weekdays.hdf  Occu_2015_8_W_Weekdays.hdf
Occu_2015_52_E_Weekends.hdf  Occu_2015_8_W_Weekends.hdf
Occu_2015_52_W_Weekdays.hdf  Occu_2015_905_E_Weekdays.hdf
Occu_2015_52_W_Weekends.hdf  Occu_2015_905_E_Weekends.hdf
Occu_2015_54_E_Weekdays.hdf  Occu_2015_905_W_Weekdays.hdf
Occu_2015_54_E_Weekends.hdf  O

In [ ]:
# create average year dataframes per direction
# Note: for some reason doing the time calculation on the entire year at once never finished, but
# breaking it into directions of N, S, E, W and then doing it worked
for Dir in ['N', 'S', 'E', 'W']:
    sub_frame = df_core[df_core.Direction == Dir]
    new_time = pd.to_datetime(sub_frame['Timestamp'], format='%m/%d/%Y %H:%M:%S').dt.time
    sub_frame['Time'] = new_time
    for col, f_name in [('Total Flow', 'Flow'), ('Avg Occupancy', 'Occu'), ('Avg Speed', 'Speed')]:
        pivoted = sub_frame.pivot_table(index='Station', columns='Time', values=col, aggfunc=np.mean)
        name = '%s_%s_2015' % (f_name, Dir)
        pivoted.to_hdf('%s.hdf' % (name), name, mode='w')

In [ ]:
# construct yearly average Flow Speed and Occu vectors from the N, S, E, W
for data_type in ['Flow', 'Speed', 'Occu']:
    files = !ls Flow_*_2015.hdf
    frames = []
    for item in files:
        name = item.split('.')[0]
        frames.append(pd.read_hdf(item, name))
    total = pd.concat(frames)
    total_mean = total.mean()
    total_mean.to_hdf('%s_2015.hdf' % data_type, '%s_2015' % data_type)

Flow_2015.hdf                Flow_2015_5_S_Weekdays.hdf
Flow_2015_125_N_Weekdays.hdf Flow_2015_5_S_Weekends.hdf
Flow_2015_125_N_Weekends.hdf Flow_2015_78_E_Weekdays.hdf
Flow_2015_125_S_Weekdays.hdf Flow_2015_78_E_Weekends.hdf
Flow_2015_125_S_Weekends.hdf Flow_2015_78_W_Weekdays.hdf
Flow_2015_15_N_Weekdays.hdf  Flow_2015_78_W_Weekends.hdf
Flow_2015_15_N_Weekends.hdf  Flow_2015_805_N_Weekdays.hdf
Flow_2015_15_S_Weekdays.hdf  Flow_2015_805_N_Weekends.hdf
Flow_2015_15_S_Weekends.hdf  Flow_2015_805_S_Weekdays.hdf
Flow_2015_163_N_Weekdays.hdf Flow_2015_805_S_Weekends.hdf
Flow_2015_163_N_Weekends.hdf Flow_2015_8_E_Weekdays.hdf
Flow_2015_163_S_Weekdays.hdf Flow_2015_8_E_Weekends.hdf
Flow_2015_163_S_Weekends.hdf Flow_2015_8_W_Weekdays.hdf
Flow_2015_52_E_Weekdays.hdf  Flow_2015_8_W_Weekends.hdf
Flow_2015_52_E_Weekends.hdf  Flow_2015_905_E_Weekdays.hdf
Flow_2015_52_W_Weekdays.hdf  Flow_2015_905_E_Weekends.hdf
Flow_2015_52_W_Weekends.hdf  Flow_2015_905_W_Weekdays.hdf
Flow_2015_54_E_Weekdays.hdf  F